In [32]:
with open("the-verdict.txt" , "r" , encoding="utf-8") as f:
    raw_text = f.read()

print("Total no. of. charachters" , len(raw_text))
print(raw_text[:99])

Total no. of. charachters 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [33]:
import re

text = "Hello world ! , This is a test"
result = re.split(r'(\s)' , text)

print(result)

['Hello', ' ', 'world', ' ', '!', ' ', ',', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test']


In [34]:
result = re.split(r'([,.;:?_!"()\']|--|\s)' , text)
print(result)

['Hello', ' ', 'world', ' ', '', '!', '', ' ', '', ',', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test']


In [35]:
## removing the white spaces
result = [item for item in result if item.strip()]
print(result)

['Hello', 'world', '!', ',', 'This', 'is', 'a', 'test']


In [36]:
preprocessed = re.split(r'([,.;:?_!"()\']|--|\s)' , raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [37]:
print(len(preprocessed))

4690


In [38]:
### sort all words in alfabetical order 
## set keyword is used to take only unique words (non-repeating)
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [39]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [40]:
## Let's print some of them to understand 

for i,item in enumerate(vocab.items()):
    print(item)
    if(i>=20):
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [41]:
### THE ABOVE PROCESS IS ENCODER 
### reversal of this process is called DECODER

In [42]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int = vocab  ##mentioned above list
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.;:?_!"()\']|--|\s)' , text)
        preprocessed = [ item.strip() for item in preprocessed if item.strip() ]
        ids = [ self.str_to_int[s] for s in preprocessed ]
        return ids

    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        ## Replace spaces before special charachters
        text = re.sub(r'\s+([,.?!"()\'])' , r'\1' , text)
        return text        

In [43]:
### Let's test it

tokenizer = SimpleTokenizer(vocab)
text = """"It's the last he painted you know,"
        Mrs. Gisburn said with pardonable pride.""" 

In [44]:
### Test for encode function whether it convert these text into ids and returning it 
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [45]:
tokenizer.decode(ids)

'" It\' s the last he painted you know," Mrs. Gisburn said with pardonable pride.'

In [46]:
### What if we add text that is not in book ?

tokenizer = SimpleTokenizer(vocab)
text = "Let's have tea ?"

In [47]:
ids = tokenizer.encode(text)
print(ids)

KeyError: 'Let'

In [53]:
### See the above wont't work as these keywords are not in book
### So , Let's add some more tokens like unknown , end of text .

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [54]:
len(vocab)

1132

In [55]:
## Let's print last 5 words in vocab
for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [56]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab  ##mentioned above list
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.;:?_!"()\']|--|\s)' , text)
        preprocessed = [ item.strip() for item in preprocessed if item.strip() ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [ self.str_to_int[s] for s in preprocessed ]
        return ids

    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        ## Replace spaces before special charachters
        text = re.sub(r'\s+([,.?!"()\'])' , r'\1' , text)
        return text        

In [57]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlight terraces of the palace. "

text = "<|endoftext|>".join((text1,text2))

print(text)

Hello, do you like tea?<|endoftext|>In the sunlight terraces of the palace. 


In [58]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1131, 988, 1131, 984, 722, 988, 1131, 7]

In [59]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|unk|> the <|unk|> terraces of the <|unk|>.'

In [1]:
### Let's use tiktoken from python , which is used by chatGPT as tokenizer

! pip3 install tiktoken

   ---------------------------------------- 0.0/884.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/884.3 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/884.3 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/884.3 kB 1.1 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/884.3 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 884.3/884.3 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import importlib
import tiktoken

print("tiktoken version" , importlib.metadata.version("tiktoken"))

tiktoken version 0.11.0


In [4]:
### This is the Byte pair tokenizer from GPT

tokenizer = tiktoken.get_encoding("gpt2")

In [5]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownplace."   
       )

integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 5372, 13]


In [6]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownplace.
